In [58]:
#%%
import math

'''
Input some parameters 
such as PIM array size, network.
'''

array = [512, 512]


network = 'Resnet18'

if network == 'VGG13' :
  image = [224, 224, 112, 112, 56, 56, 28, 28, 14, 14]
  kernel = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
  channel = [3, 64, 64, 128, 128, 256, 256, 512, 512, 512, 512]

elif network == 'Resnet18' :
  image = [112, 56, 28, 14, 7]
  kernel = [7, 3, 3, 3, 3]
  channel = [3, 64, 128, 256, 512]

elif network == 'Resnet20':
  image = [32, 32, 16, 8]
  kernel = [3, 3, 3, 3]
  channel = [3, 16, 32, 64]

# %%
def im2col (image_col, image_row, filter_col, filter_row, in_channel, out_channel, array_row, array_col) :

    col_slide = image_col - filter_col + 1
    row_slide = image_row - filter_row + 1
    
    col_cycle = math.ceil(out_channel/array_col)
    row_cycle = math.ceil(filter_row*filter_col*in_channel/array_row)
    total_cycle = col_slide * row_slide * row_cycle * col_cycle
    total_datasize = col_slide*row_slide*filter_row*filter_col*in_channel
    
    return total_cycle, total_datasize


def SDK (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :
    
    row_vector = filter_row * filter_col * in_channel
    col_vector = out_channel
    
    used_row = math.ceil(row_vector/array_row)
    used_col = math.ceil(col_vector/array_col)
    
    new_array_row = array_row * used_row
    new_array_col = array_col * used_col

    # initialize
    cycle = []
    w = []
    w.append(filter_row*filter_col)
    cycle.append(used_row*used_col*(image_row-filter_row+1)*(image_col-filter_col+1))
    
    i=0
    while True :
        i += 1
        pw_row = filter_row + i - 1 
        pw_col = filter_col + i - 1
        pw = pw_row * pw_col
        if pw*in_channel <= new_array_row and i * i * out_channel <= new_array_col :
            parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            parallel_window_col = math.ceil((image_col - (filter_col + i) + 1)/i) + 1
            
            if parallel_window_row * parallel_window_row * used_row * used_col <= cycle[0] :
                del cycle[0]
                del w[0]
                cycle.append(parallel_window_row * parallel_window_col * used_row * used_col)
                w.append(pw)
                total_datasize = parallel_window_col*parallel_window_row*pw*in_channel
            
        else :
            break
        
    
    return cycle, w, total_datasize

# ceil : up, floor : down
def vw_sdk (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :

    i = 0 # initialize # overlap col
    j = 1 # overlap row

    reg_total_cycle = [] # initialize
    reg_overlap_row = []
    reg_overlap_col = []
    reg_row_cycle = []
    reg_col_cycle = []
    reg_ICt = []
    reg_OCt = []

    while True :
        try :
            i += 1
            if (i + filter_col) > image_col: 
                i = 1
                j += 1
                if j + filter_row > image_row : 
                    break
            # for parallel_window computing
            reg_N_parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            reg_N_parallel_window_col = math.ceil((image_col - (filter_col + j) + 1)/j) + 1
            
            # for cycle computing
            # Tiled IC
            if in_channel == 3 :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                if ICt > in_channel :
                    ICt = 3
                row_cycle = math.ceil(in_channel / ICt)
            else :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                row_cycle = math.ceil(in_channel / ICt)
            
            # Tiled OC
            OCt =  math.floor(array_col / (i * j))
            col_cycle = math.ceil(out_channel / OCt)
    
            reg_N_of_computing_cycle = reg_N_parallel_window_row * reg_N_parallel_window_col \
                                    * row_cycle * col_cycle
 
            if i == 1 : # initialize
                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)
      

            if reg_total_cycle[0] > reg_N_of_computing_cycle :
                del reg_total_cycle[0]
                del reg_overlap_row[0]
                del reg_overlap_col[0]
                del reg_row_cycle[0]
                del reg_col_cycle[0]
                del reg_ICt[0]
                del reg_OCt[0]

                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)
                
      
        except ZeroDivisionError :
            continue

    return reg_total_cycle[0], reg_overlap_col[0], reg_overlap_row[0], reg_row_cycle[0], reg_col_cycle[0], reg_ICt[0], reg_OCt[0]

In [82]:
import csv
import os
from typing import List, Union

def combine_arrays_to_csv(
    array1: List[List[Union[float, int]]],
    array2: List[List[Union[float, int]]],
    filename: str = "combined_output.csv"
) -> None:
    
    # 验证输入数组长度
    if len(array1) != len(array2):
        raise ValueError("输入数组必须具有相同的行数")
    
    # 合并数据
    combined_data = []
    for row1, row2 in zip(array1, array2):
        combined_row = row1 + row2
        combined_data.append(combined_row)
    
    # 写入CSV文件
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(combined_data)
        
    print(f"文件已保存至 {filename}")
    print(f"生成数据：{len(combined_data)} 行，每行 {len(combined_data[0])} 列")



def reproduce_exp(NetWork, array_row, array_col):
    net, conv, fc = read_csv(NetWork)
    net_im2 = []
    net_SDK = []
    net_vwsdk = []
    net_im2pw = []
    net_SDKpw = []
    net_vwsdkpw = []
    net_fm = []
    net_fmpw = []

    for i in range(len(net)):
        if net[i][3]!=1:

            T_im2col, data_im2 = im2col(net[i][0], net[i][1], net[i][3], net[i][3], net[i][2], net[i][5],array_row, array_col)
            T_SDK, w, data_SDK = SDK(net[i][0], net[i][1], net[i][3], net[i][3], net[i][2], net[i][5],array_row, array_col)
            total_cycle, overlap_col, overlap_row, row_cycle, col_cycle, ICt, OCt = vw_sdk(net[i][0], net[i][1], net[i][3], net[i][3], net[i][2], net[i][5],array_row, array_col)
            
            vwc_array = row_cycle*col_cycle
            pw_row = net[i][3]+overlap_row-1
            pw_col = net[i][3]+overlap_col-1

            im2_perf, im2_pw = calculate_performance(net[i][0],net[i][1],net[i][3],net[i][2],net[i][5],net[i][3],net[i][3],net[i][2],net[i][5],array_row,array_col)
            SDK_perf, SDK_pw = calculate_performance(net[i][0],net[i][1],net[i][3],net[i][2],net[i][5],math.sqrt(w[0]),math.sqrt(w[0]),net[i][2],net[i][5],array_row,array_col)
            vw_sdk_perf, vw_sdk_pw = calculate_performance(net[i][0],net[i][1],net[i][3],net[i][2],net[i][5],pw_row,pw_col,ICt,OCt,array_row,array_col)
            auto_mm = auto_mapping(net[i][0],net[i][1],net[i][3],net[i][2],net[i][5],array_row,array_col,vwc_array)
            
            net_im2.append(im2_perf)
            net_im2pw.append(im2_pw)
            net_SDK.append(SDK_perf)
            net_SDKpw.append(SDK_pw)
            net_vwsdk.append(vw_sdk_perf)
            net_vwsdkpw.append(vw_sdk_pw)
            net_fm.append([auto_mm[0],auto_mm[1],auto_mm[2]])
            net_fmpw.append(auto_mm[3])

        else:
            fc_perf, fc_pw = calculate_performance_fc(net[i][0],net[i][1],net[i][3],net[i][2],net[i][5],array_row,array_col)
            net_im2.append(fc_perf)
            net_im2pw.append(fc_pw)
            net_SDK.append(fc_perf)
            net_SDKpw.append(fc_pw)
            net_vwsdk.append(fc_perf)
            net_vwsdkpw.append(fc_pw)
            net_fm.append(fc_perf)
            net_fmpw.append(fc_pw)
    
    name = os.path.splitext(NetWork)[0]
    combine_arrays_to_csv(net_im2,net_im2pw,name+"im2.csv")
    combine_arrays_to_csv(net_SDK,net_SDKpw,name+"SDK.csv")
    combine_arrays_to_csv(net_vwsdk,net_vwsdkpw,name+"vwsdk.csv")
    return net_im2,net_im2pw,net_SDK,net_SDKpw,net_vwsdk,net_vwsdkpw,net_fm,net_fmpw

文件已保存至 example_output.csv
生成数据：3 行，每行 7 列


In [78]:
   
a,b,c,d,e,f,g = vw_sdk(32,32,3,3,64,64,512,512)
# SDK(224,224,3,3,64,64,512,512)
# read_csv("NetWork_Resnet20.csv")[0][0][5]

In [90]:
net_im2,net_im2pw,net_SDK,net_SDKpw,net_vwsdk,net_vwsdkpw=reproduce_exp("NetWork_Resnet20.csv",512,512)
# print(net_im2)
# print(net_im2pw)
# print(net_SDK)
# print(net_SDKpw)
print(net_vwsdk)
print(net_vwsdkpw)

文件已保存至 NetWork_Resnet20im2.csv
生成数据：20 行，每行 7 列
文件已保存至 NetWork_Resnet20SDK.csv
生成数据：20 行，每行 7 列
文件已保存至 NetWork_Resnet20vwsdk.csv
生成数据：20 行，每行 7 列
[[6120.0, 30, 1], [28800.0, 60, 2], [28800.0, 60, 2], [28800.0, 60, 2], [28800.0, 60, 2], [28800.0, 60, 2], [28800.0, 60, 2], [38400.0, 75, 1], [20400.0, 40, 2], [20400.0, 40, 2], [20400.0, 40, 2], [20400.0, 40, 2], [20400.0, 40, 2], [25088.0, 49, 1], [9216.0, 18, 2], [9216.0, 18, 2], [9216.0, 18, 2], [9216.0, 18, 2], [9216.0, 18, 2], [64.0, 1.0, 1]]
[[17.0, 4.0, 3, 17], [12.0, 5.0, 8, 17], [12.0, 5.0, 8, 17], [12.0, 5.0, 8, 17], [12.0, 5.0, 8, 17], [12.0, 5.0, 8, 17], [12.0, 5.0, 8, 17], [8.0, 4.0, 16, 42], [6.0, 5.0, 17, 42], [6.0, 5.0, 17, 42], [6.0, 5.0, 17, 42], [6.0, 5.0, 17, 42], [6.0, 5.0, 17, 42], [4.0, 4.0, 32, 128], [4.0, 4.0, 32, 128], [4.0, 4.0, 32, 128], [4.0, 4.0, 32, 128], [4.0, 4.0, 32, 128], [4.0, 4.0, 32, 128], [1.0, 1.0, 64.0, 10.0]]


In [108]:
import math
# Complete the performance calculation of the fully connected layer
def calculate_performance_fc(image_row, image_col, kernel, inchannel, outchannel,array_row, array_col):

    # Calculate the transmission cycle and the amount of data transmitted
    row = math.ceil(inchannel/array_row)
    col = math.ceil(outchannel/array_col)
    total_tranferdata = image_row*image_col*inchannel
    total_computecycle = row*col*image_col*image_row
    total_array = row*col

    # Computational performance
    performance = []
    performance.append(total_tranferdata)
    performance.append(total_computecycle)
    performance.append(total_array)

    # Calculation window
    flexwindow = []
    flexwindow.append(kernel)
    flexwindow.append(kernel)
    flexwindow.append(inchannel)
    flexwindow.append(outchannel)
    
    return performance, flexwindow


# conv calculate conpute cycle and transmit data
def calculate_performance(image_row, image_col, kernel, inchannel, outchannel, PW_h, PW_w, ICt, OCt, array_row, array_col):
    # reg transmit data
    flexible_window = []
    performance = []
    i = PW_h-kernel+1
    j = PW_w-kernel+1
    row_outCount = math.ceil((image_row-PW_h)/(PW_h-kernel+1))+1
    col_outCount = math.ceil((image_col-PW_w)/(PW_w-kernel+1))+1
    total_transfer = row_outCount*col_outCount
    num_ic = math.ceil(inchannel/ICt)
    # num_oc = math.ceil(outchannel/OCt)
    totalTransferredData = total_transfer*PW_h*PW_w*min(ICt*num_ic,inchannel)
    #reg compute cycle
    AR_cycle = math.ceil(inchannel*PW_h*PW_w/array_row)
    AC_cycle = math.ceil(OCt*i*j/array_col)*math.ceil(outchannel/OCt)
    total_compute_cycle = total_transfer*AR_cycle*AC_cycle
    #reg array num
    flexible_window.append(PW_h)
    flexible_window.append(PW_w)
    flexible_window.append(min(ICt,math.ceil(inchannel/num_ic)))
    flexible_window.append(OCt)
    total_array = AC_cycle*AR_cycle

    performance.append(totalTransferredData)
    performance.append(total_compute_cycle)
    performance.append(total_array)
    return performance, flexible_window

#
def filter_by_index_and_value(data, index, value):
    """
    Filter subarrays in a two-dimensional array based on specified indexes and values
    : param data:  Two-dimensional array
    : param index:  Index to be filtered
    : param value:  Target value (can be a single value or a list)
    : return:  List of subarrays that meet the criteria
    """
    result = []
    for item in data:
        # Check if the index is valid
        if index < 0 or index >= len(item):
            continue  # Skip subarrays with invalid indexes
        if isinstance(value, list):
            if item[index] in value:
                result.append(item)
        else:
            if item[index] == value:
                result.append(item)
    return result
def remove_duplicates(data):
    """
    Remove duplicate subarrays from a two-dimensional array
    : param data:  Two-dimensional array
    : return:  List without duplicate subarrays
    """
    unique_items = []
    seen_features = set()
    
    for item in data:
        # Extract the first three elements
        prefix = tuple(item[:3])
        
        # Extract the combination of the first two elements from the fourth list (sorted)
        fourth_list = item[3]
        if len(fourth_list) >= 2:
            sorted_pair = tuple(sorted(fourth_list[:2]))
        else:
            sorted_pair = tuple(fourth_list)
        
        # Extract the last two elements of the fourth list
        suffix = tuple(fourth_list[2:]) if len(fourth_list) >= 2 else tuple()
        
        # Combined characteristics
        feature = (prefix, sorted_pair, suffix)
        
        if feature not in seen_features:
            seen_features.add(feature)
            unique_items.append(item)
    
    return unique_items


def select_design_by_mode(design: dict, resource_limit: float, mode: str = "compute_first", alpha: float = 0.5):
    import sys

    layers = sorted(design.keys(), key=lambda x: int(x.split('layer')[-1]))
    layer_choices = [design[layer] for layer in layers]

    best_selection = None
    best_metrics = (float('inf'), float('inf'), float('inf'))

    # 预处理计算所有可行路径的最大/最小值，用于归一化
    all_compute_vals, all_cycle_vals = [], []

    def gather_extremes():
        def dfs_ext(i, total_compute, total_cycle, total_resource):
            if total_resource > resource_limit:
                return
            if i == len(layers):
                all_compute_vals.append(total_compute)
                all_cycle_vals.append(total_cycle)
                return
            for opt in layer_choices[i]:
                dfs_ext(i + 1, total_compute + opt[0], total_cycle + opt[1], total_resource + opt[2])
        dfs_ext(0, 0.0, 0, 0.0)

    gather_extremes()

    compute_min, compute_max = min(all_compute_vals), max(all_compute_vals)
    cycle_min, cycle_max = min(all_cycle_vals), max(all_cycle_vals)

    def normalize(val, min_val, max_val):
        return 0.0 if max_val == min_val else (val - min_val) / (max_val - min_val)

    def compute_stability(cycles):
        return sum(abs(cycles[i] - cycles[i+1]) for i in range(len(cycles)-1))

    def dfs(layer_index, current_selection, total_resource, total_cycle, total_compute, cycle_list):
        nonlocal best_selection, best_metrics

        if total_resource > resource_limit:
            return
        if layer_index == len(layers):
            stability = compute_stability(cycle_list)

            # Normalize values for fair weighting
            norm_compute = normalize(total_compute, compute_min, compute_max)
            norm_cycle = normalize(total_cycle, cycle_min, cycle_max)

            if mode == "compute_first":
                metrics = (total_compute, stability, total_cycle)
            elif mode == "cycle_first":
                metrics = (total_cycle, stability, total_compute)
            elif mode == "balanced":
                weighted_score = alpha * norm_compute + (1 - alpha) * norm_cycle
                metrics = (weighted_score, stability, total_cycle)
            else:
                raise ValueError("Unsupported mode")

            if metrics < best_metrics:
                best_metrics = metrics
                best_selection = list(current_selection)
            return

        for option in layer_choices[layer_index]:
            compute = option[0]
            cycle = option[1]
            resource = option[2]

            current_selection.append(option)
            dfs(layer_index + 1,
                current_selection,
                total_resource + resource,
                total_cycle + cycle,
                total_compute + compute,
                cycle_list + [cycle])
            current_selection.pop()

    dfs(0, [], 0.0, 0, 0.0, [])

    if best_selection is None:
        return {}

    return {layer: best_selection[i] for i, layer in enumerate(layers)}



a=calculate_performance(112,112,7,3,64,10,8,3,64,512,512)
a

([343440, 1431, 1], [10, 8, 3, 64])

In [111]:
import csv
# Read neural network data
# Split convolutional layers and fully connected layers
def read_csv(file_path):
    conv_layers = []
    fc_layers = []
    netstructure = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            float_row = [float(value) for value in row]
            if float_row[3]==1:
                float_row.append(0)
                fc_layers.append(float_row)   
            else:
                float_row.append(1)
                conv_layers.append(float_row)
            netstructure.append(float_row)
    return netstructure, conv_layers,fc_layers

# Computational neural networks require arrays
def calculate_min_array(file_path, array_row, array_col):
    net, conv, fc = read_csv(file_path)
    conv_array = 0
    fc_array = 0
    for i in range(len(net)):
        row = net[i][2]*net[i][3]*net[i][4]
        col = net[i][5]
        col_num = math.ceil(row/array_row)
        row_num = math.ceil(col/array_col)
        net[i].append(col_num*row_num)
    for i in range(len(conv)):
        row = conv[i][2]*conv[i][3]*conv[i][4]
        col = conv[i][5]
        conv_row_num = math.ceil(row/array_row)
        conv_col_num = math.ceil(col/array_col)
        conv_array = conv_row_num*conv_col_num + conv_array
    for j in range(len(fc)):
        fc_row_num = math.ceil(fc[j][2]/array_row)
        fc_col_num = math.ceil(fc[j][5]/array_col)
        fc_array = fc_array + fc_col_num*fc_row_num
    return net, conv_array, fc_array



#Split the data in the list
def split_listdata(data_list, index):
    data1 = []
    data2 = []
    for item in data_list:
        data1.append(item[:index])
        data2.append(item[index:])
    return data1, data2

# pareto
def is_dominated(solution1, solution2):
        """
        判断solution1是否被solution2支配
        :param solution1: 第一个解
        :param solution2: 第二个解
        :return: 如果solution1被solution2支配,返回True,否则返回False
        """
        # 检查solution2是否在所有指标上都优于solution1
        # for i in range(len(solution1)):
        if (solution2[0] <= solution1[0] and
            solution2[1] <= solution1[1] and
            solution2[2] <= solution1[2]):
            # 至少有一个指标更优
            if (solution2[0] < solution1[0] or
                solution2[1] < solution1[1] or
                solution2[2] < solution1[2]):
                return True
        return False

def update_pareto_set(new_solution, pareto_set):
    """
    更新帕累托最优解集
    :param new_solution: 新计算出的解（字典形式，包含三个指标）
    :param pareto_set: 当前的帕累托最优解集（列表形式）
    :return: 更新后的帕累托最优解集
    """
    # 检查新解是否被现有解集中的解支配
    dominated_by_existing = False
    
    if len(pareto_set)==0:
        pareto_set.append(new_solution)
        return pareto_set
    else:
        # filtered_data = [item for item in pareto_set if item[2] == resource]
        for solution in pareto_set:
            # if solution[2] == resource:
                if is_dominated(new_solution, solution):
                    dominated_by_existing = True
                    break
            # else:
            #     continue
        
        if not dominated_by_existing:
            # 移除被新解支配的解
            new_pareto_set = []
            for solution in pareto_set:
                # if solution[2] == resource:
                    if not is_dominated(solution, new_solution):
                        new_pareto_set.append(solution)
                # else:
                #     continue
            new_pareto_set.append(new_solution)
            return new_pareto_set
        else:
            return pareto_set

def auto_mapping(image_row, image_col, kernel, inchannel, outchannel, array_row, array_col, array_limit):
    PW_h = kernel
    PW_w = kernel
    # reg pareto set
    s1 = []
    s2 = []
    pareto = []
    pareto_performance = []
    pareto_design = []
    flex_window = []
    total_pareto = []
    s1_fc = []
    s2_fc = []
    # fc layer
    if kernel == 1:
        s1_fc, flex_window = calculate_performance_fc(image_row, image_row, kernel, inchannel, outchannel, array_row, array_col)
        s1_fc.append(flex_window)

        return [s1_fc]
    # conv layer
    else:
        for i in range(image_row - kernel):
            for j in range(image_col - kernel):
                PW_h = kernel + i
                PW_w = kernel + j
                ICt = max(math.floor(array_row/PW_h/PW_w),1)
                OCt = max(math.floor(array_col/(i+1)/(j+1)),1)
                if OCt>=outchannel:
                    OCt = outchannel
                if ICt>=inchannel:
                    ICt = inchannel
                s1, flex_window = calculate_performance(image_row, image_row, kernel, inchannel, outchannel,PW_h, PW_w, ICt, OCt, array_row, array_col)
                # s1.append(total_transferdata)
                # s1.append(total_computecycle)
                # s1.append(total_array)
                s1.append(flex_window)
                s2 = s1[:]
                # pareto.append(s2)
                pareto = update_pareto_set(s2,pareto)
                
                # pareto_performance, pareto_design = split_listdata(pareto, 3)
                s1.clear()
                # print("The total amount of data transmitted :",total_transferdata)
                # print("Total calculation cycle :",total_computecycle)
                # print("The total array :",total_array)
        for j in range(1,array_limit+1):
            pareto1 = filter_by_index_and_value(pareto,2,j)
            pareto1 = remove_duplicates(pareto1)
            total_pareto = total_pareto + pareto1
        return total_pareto #pareto_performance,pareto_design


def Automated_Exploration_Framework(Network, array_row, array_col, max_numarray, mode, alpha: float = 0.5):
    net,conv_array, fc_array = calculate_min_array(Network,array_row,array_col)
    minnum_array = conv_array + fc_array
    # If it is smaller than the minimum array, special mapping is required
    # If greater than or equal to the minimum array, automatic mapping search is required
    net_array = []

    # result = []
    variable_array = 0
    for j in range(len(net)):
        net_array.append(net[j][9])
        if net[j][9]!=1:
            variable_array = variable_array + 1
    net_design = {}
    if minnum_array <= max_numarray:
        remain_array = max_numarray - minnum_array
        for i in range(len(net)):
            if net[i][8] == 1:
                perf = auto_mapping(int(net[i][0]),int(net[i][1]),int(net[i][3]),\
                    net[i][2],net[i][5],array_row,array_col,net[i][9]+remain_array)
                net_design['conv_layer'+str(i)] = perf
            else:
                perf = auto_mapping(int(net[i][0]),int(net[i][1]),int(net[i][3]),\
                    net[i][2],net[i][5],array_row,array_col,net[i][9]+remain_array)
                net_design['fc_layer'+str(i)] = perf
            # print(perf)
        if mode == "compute_first":
            result = select_design_by_mode(net_design, max_numarray, mode)
        elif mode == "cycle_first":
            result = select_design_by_mode(net_design, max_numarray, mode)
        else:
            result = select_design_by_mode(net_design, max_numarray, mode, alpha)
    elif minnum_array-variable_array<=max_numarray<minnum_array:
        print("------------Special mapping-----------------")
    else:
        print("------------The number of storage arrays is too small, please reset--------")
        print("------------The minimum required storage array is:",minnum_array)

    return result


    # else:

performance=auto_mapping(32,32,3,16,16,512,512,2)
# design = Automated_Exploration_Framework('NetWork_Resnet20.csv',512,512,30,"compute_first")
print(performance)
# print(design)
# performance

[[38400, 75, 1, [4, 8, 16, 16]], [26880, 60, 2, [7, 8, 8, 16]]]


In [98]:
auto_mapping(32,32,3,16,16,512,512,2)

[[38400, 75, 1, [4, 8, 16, 16]], [28800, 60, 2, [5, 12, 8, 16]]]

In [96]:
calculate_performance(32,32,3,16,16,5,12,8,16,512,512)

([28800, 60, 2], [5, 12, 8, 16])

In [97]:
calculate_performance(32,32,3,16,16,7,8,8,16,512,512)

([26880, 60, 2], [7, 8, 8, 16])

In [21]:
result = filter_by_index_and_value(performance,2,3)
remove_duplicates(result)

[[216240, 1590, 3, [8, 17, 3, 23]],
 [202176, 1728, 3, [9, 13, 3, 24]],
 [196020, 1782, 3, [10, 11, 3, 25]]]

In [16]:
performance

[[343440, 1431, 1, [8, 10, 3, 64]],
 [216240, 1590, 3, [8, 17, 3, 23]],
 [202176, 1728, 3, [9, 13, 3, 24]],
 [343440, 1431, 1, [10, 8, 3, 64]],
 [218700, 1458, 2, [10, 10, 3, 32]],
 [196020, 1782, 3, [10, 11, 3, 25]],
 [158760, 1512, 4, [10, 14, 3, 16]],
 [126360, 3564, 22, [10, 26, 1, 6]],
 [196020, 1782, 3, [11, 10, 3, 25]],
 [156816, 1584, 4, [11, 12, 3, 17]],
 [130680, 1584, 6, [11, 15, 3, 11]],
 [156816, 1584, 4, [12, 11, 3, 17]],
 [139968, 1620, 5, [12, 12, 3, 14]],
 [127008, 1764, 7, [12, 14, 3, 10]],
 [99792, 3276, 26, [12, 22, 1, 5]],
 [202176, 1728, 3, [13, 9, 3, 24]],
 [158760, 1512, 4, [14, 10, 3, 16]],
 [127008, 1764, 7, [14, 12, 3, 10]],
 [130680, 1584, 6, [15, 11, 3, 11]],
 [122400, 3120, 26, [15, 17, 2, 5]],
 [86400, 3072, 32, [15, 20, 1, 4]],
 [77760, 4752, 66, [15, 24, 1, 3]],
 [85536, 3168, 32, [16, 18, 1, 4]],
 [216240, 1590, 3, [17, 8, 3, 23]],
 [122400, 3120, 26, [17, 15, 2, 5]],
 [81600, 3520, 44, [17, 20, 1, 3]],
 [85536, 3168, 32, [18, 16, 1, 4]],
 [78732, 3564

In [41]:
net,conv_array, fc_array = calculate_min_array("NetWork_Resnet20.csv",512,512)
net

[[32.0, 32.0, 3.0, 3.0, 3.0, 16.0, 0.0, 0.0, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.125, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.1875, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.28125, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.21875, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.21875, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 16.0, 0.0, 0.28125, 1, 1],
 [32.0, 32.0, 16.0, 3.0, 3.0, 32.0, 1.0, 0.125, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 32.0, 0.0, 0.132812, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 32.0, 0.0, 0.421875, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 32.0, 0.0, 0.148438, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 32.0, 0.0, 0.21875, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 32.0, 0.0, 0.210938, 1, 1],
 [16.0, 16.0, 32.0, 3.0, 3.0, 64.0, 1.0, 0.164062, 1, 1],
 [8.0, 8.0, 64.0, 3.0, 3.0, 64.0, 0.0, 0.191406, 1, 2],
 [8.0, 8.0, 64.0, 3.0, 3.0, 64.0, 0.0, 0.287109, 1, 2],
 [8.0, 8.0, 64.0, 3.0, 3.0, 64.0, 0.0, 0.376953, 1, 2],
 [8.0, 8.0, 64.0, 3.0, 3.0, 64.0, 0.0, 

In [74]:
calculate_min_array('NetWork_Resnet20.csv',512,512)

(24, 1)

In [76]:
conv, fc=read_csv('NetWork_Resnet20.csv')
fc

[[1.0, 1.0, 64.0, 1.0, 1.0, 10.0, 0.0, 0.0]]

In [96]:
# 模式一：优先考虑计算量最小
result1 = select_design_by_mode(design, resource_limit=30, mode="compute_first")

# 模式二：优先考虑周期最短
result2 = select_design_by_mode(design, resource_limit=30, mode="cycle_first")
print(result1)
print(result2)

{'conv_layer0': [5040.0, 30, 1, [7, 8, 3.0, 16.0]], 'conv_layer1': [28800, 60, 2, [5, 12, 8, 16.0]], 'conv_layer2': [28800, 60, 2, [5, 12, 8, 16.0]], 'conv_layer3': [28800, 60, 2, [5, 12, 8, 16.0]], 'conv_layer4': [28800, 60, 2, [5, 12, 8, 16.0]], 'conv_layer5': [28800, 60, 2, [5, 12, 8, 16.0]], 'conv_layer6': [38400.0, 75, 1, [4, 8, 16.0, 16.0]], 'conv_layer7': [38400.0, 75, 1, [4, 8, 16.0, 32.0]], 'conv_layer8': [25088.0, 49, 1, [4, 4, 32.0, 32.0]], 'conv_layer9': [25088.0, 49, 1, [4, 4, 32.0, 32.0]], 'conv_layer10': [25088.0, 49, 1, [4, 4, 32.0, 32.0]], 'conv_layer11': [25088.0, 49, 1, [4, 4, 32.0, 32.0]], 'conv_layer12': [25088.0, 49, 1, [4, 4, 32.0, 32.0]], 'conv_layer13': [25088.0, 49, 1, [4, 4, 32.0, 64.0]], 'conv_layer14': [9216, 18, 2, [4, 4, 32, 64.0]], 'conv_layer15': [9216, 18, 2, [4, 4, 32, 64.0]], 'conv_layer16': [9216, 18, 2, [4, 4, 32, 64.0]], 'conv_layer17': [9216, 18, 2, [4, 4, 32, 64.0]], 'conv_layer18': [9216, 18, 2, [4, 4, 32, 64.0]], 'fc_layer19': [64.0, 1, 1, [1,

In [11]:
result3 = select_design_by_mode(design, resource_limit=30, mode="balanced", alpha=0.6)
result3

{'conv_layer0': [5040.0, 30, 1, [7, 8, 3.0, 16.0]],
 'conv_layer1': [28800, 60, 2, [5, 12, 8, 16.0]],
 'conv_layer2': [28800, 60, 2, [5, 12, 8, 16.0]],
 'conv_layer3': [28800, 60, 2, [5, 12, 8, 16.0]],
 'conv_layer4': [28800, 60, 2, [5, 12, 8, 16.0]],
 'conv_layer5': [28800, 60, 2, [5, 12, 8, 16.0]],
 'conv_layer6': [38400.0, 75, 1, [4, 8, 16.0, 16.0]],
 'conv_layer7': [38400.0, 75, 1, [4, 8, 16.0, 32.0]],
 'conv_layer8': [25088.0, 49, 1, [4, 4, 32.0, 32.0]],
 'conv_layer9': [25088.0, 49, 1, [4, 4, 32.0, 32.0]],
 'conv_layer10': [25088.0, 49, 1, [4, 4, 32.0, 32.0]],
 'conv_layer11': [25088.0, 49, 1, [4, 4, 32.0, 32.0]],
 'conv_layer12': [25088.0, 49, 1, [4, 4, 32.0, 32.0]],
 'conv_layer13': [25088.0, 49, 1, [4, 4, 32.0, 64.0]],
 'conv_layer14': [9216, 18, 2, [4, 4, 32, 64.0]],
 'conv_layer15': [9216, 18, 2, [4, 4, 32, 64.0]],
 'conv_layer16': [9216, 18, 2, [4, 4, 32, 64.0]],
 'conv_layer17': [9216, 18, 2, [4, 4, 32, 64.0]],
 'conv_layer18': [9216, 18, 2, [4, 4, 32, 64.0]],
 'fc_layer19